In [1]:
from google.auth.transport import requests
from googleapiclient import discovery
from google.oauth2 import service_account
from google.cloud import storage
from googleapiclient.http import MediaIoBaseDownload
import io
import threading
from time import sleep
import pandas as pd

In [2]:
ls = []
def f1():
    i= 0
    while True:
        i += 1
        ls.append(i)
        time.sleep(5)

In [3]:

t1 = threading.Thread(target=f1)
t1.start()

In [3]:

def get_client():
    # Google Drive
    credentials_drive = service_account.Credentials.from_service_account_file(
        filename='savvy-concord-374520-bbbb0e3c73c3.json',
        scopes=['https://www.googleapis.com/auth/drive'],
    )
    session = requests.AuthorizedSession(credentials_drive)
    service = discovery.build('drive', 'v3', credentials=credentials_drive)

    #Google Storage
    credentials_storage = service_account.Credentials.from_service_account_file(
        filename='credentials.json',
        scopes=['https://www.googleapis.com/auth/devstorage.full_control'],
    )
    client = storage.Client(credentials=credentials_storage)
    bucket = client.get_bucket('cabinet_bucket')

    return service, bucket

# Общие диски
def get_list_of_drives(service):
    response = service.drives().list(pageSize=100).execute()
    ids = []
    for drive in response['drives']:
        ids.append([drive['name'], drive['id']])
    return ids

def get_list_of_files_in_drive(service, drive_name_id):
    ls_files = []
    ls_files_size = []

    response = service.files().list(
        q="mimeType!='application/vnd.google-apps.folder'",
        corpora='drive',
        driveId=drive_name_id[1],
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,

        fields='nextPageToken, files(id, name, size)').execute()

    while response:
        for file in response.get('files', []):
            # s_path =get_full_path_to_file(drive_name_id, file.get('id'))
            ls_files.append(file.get('id'))
            ls_files_size.append(file.get('size'))
            # ls_files.append([file.get('name'),file.get('id')])

        page_token = response.get('nextPageToken', None)

        if page_token is None:
            break

        response = service.files().list(
            q="mimeType!='application/vnd.google-apps.folder'",
            corpora='drive',
            driveId=drive_name_id[1],
            includeItemsFromAllDrives=True,
            supportsAllDrives=True,
            fields='nextPageToken, files(id, name, size)',
            pageToken=page_token).execute()
    return ls_files, ls_files_size


# Полный путь к файлу
def get_full_path_to_file(service, drive_name_id, file_id):
    s_main_folder = 'GoogleDriveTransfer/'
    ls = get_full_path_to_file_list(service, drive_name_id[1], file_id, [])
    ls.reverse()
    ls[0] = drive_name_id[0]
    s_path = s_main_folder + '/'.join(ls)
    return s_path


def get_full_path_to_file_list(service, drive_id, file_id, ls):
    response = service.files().get(
        fileId=file_id,
        fields='id, name, parents',
        supportsAllDrives=True).execute()
    file = response
    # print(file['name'])
    ls.append(file['name'])
    if file.get('parents') is not None:
        for parent in file['parents']:
            get_full_path_to_file_list(service, drive_id, parent, ls)
    return ls

# Скачать файл
def download_file(service, bucket, drive_name_id, file_id):
    file_path = get_full_path_to_file(service, drive_name_id, file_id)

    blobs = bucket.list_blobs(prefix=file_path)
    files = [blob.name for blob in blobs]
    if file_path in files:
        # print('Файл уже существует')
        return 0

    try:
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            # print("Download %d%%." % int(status.progress() * 100))

        blob = bucket.blob(file_path)
        blob.upload_from_string(fh.getvalue())
    except:
        ls_error.append([file_path, 'Ошибка при скачивании'], e)
        return 0

    return 1

In [3]:
from datetime import time

drivers_check_list = []
drivers_checked_list = []
ls_threads = []
ls_tmp = []
ls_error = []
drivers = [['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
]

def transfer_drive(drive_name_id):
    while True:
        is_download_files = True

        try:
            service, bucket = get_client()
            ls_files = get_list_of_files_in_drive(service, drive_name_id)
            for file in ls_files:
                done = download_file(service, bucket, drive_name_id, file)
                if done == 1:
                    is_download_files = False
                # try:
                #     done = download_file(service, bucket, drive_name_id, file)
                #     if done == 1:
                #         is_download_files = False
                # except Exception as e:
                #     ls_error.append([drive_name_id, get_full_path_to_file(service, drive_name_id,file), e])

            ls_tmp.append(f'{drive_name_id[0]}  - Завершено  - {is_download_files}')
        except Exception as e:
            ls_error.append([drive_name_id, 'Ошибка', e])

        if is_download_files:
            drivers_checked_list.append(drive_name_id[1])
            break



def transfer_drive_thread():
    while True:
        try:
            service, bucket = get_client()
            ls_drives = get_list_of_drives(service)
            s_drives = 'TT '
            drivers1 = []
            for l in ls_drives:
                for d in drivers:
                    if l[0] == d[0]:
                        drivers1.append(l)

            for drive in drivers1:
                if drive[1] not in drivers_check_list and drive[1] not in drivers_checked_list:
                    drivers_check_list.append(drive[1])
                    t = threading.Thread(target=transfer_drive, args=(drive,))
                    t.start()
                    ls_threads.append(t)
                    s_drives += drive[0] + ', '
        except Exception as e:
            ls_error.append(['get_list_of_drives', e])
        ls_tmp.append(f'Завершено------------------------{s_drives}')

        try:
            sleep(60*10)
        except Exception as e:
            ls_error.append(e)



def transfer_drives_thread():
    t = threading.Thread(target=transfer_drive_thread)
    ls_threads.append(t)
    t.start()

    return t


In [4]:
t = transfer_drives_thread()

In [4]:
service, bucket = get_client()

In [ ]:
blobs = bucket.list_blobs(prefix=file_path)

In [5]:
ls_drives = get_list_of_drives(service)

In [7]:
drivers

[['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA']]

In [6]:
ls_drives

[['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Ci', '0AHj-PRcbOxSWUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Managment 2', '0AE3Z9Ox2YaurUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['VG.COM', '0AElRg0lTw8iIUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Архив', '0AOP_oKFy4oGJUk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['Винный парк', '0ANhu4lfOrlBiUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Контент_VG', '0AN6chrg4ApnnUk9PVA'],
 ['Контент', '0ANf3LpamTrsTUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
 ['Лобня', '0AIQWy4IyyeaIUk9PVA'],
 ['НКВЗ ремонт АвтоТехники', '0ADgaxv8NdpTkUk9PVA'],
 ['Объединенн

In [9]:
drivers[0][0]

'BackOffice'

In [12]:
drivers1 = []
for l in ls_drives:
    for d in drivers:
        if l[0] == d[0]:
            drivers1.append(l)
drivers1

[['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA']]

In [19]:
# объем диска
def get_size_of_drive(service, drive_name_id):
    ls_files = get_list_of_files_in_drive(service, drive_name_id)
    size = 0
    for file in ls_files:
        try:
            file_info = service.files().get(fileId=file, fields='size').execute()
            size += int(file_info.get('size'))
        except Exception as e:
            ls_error.append([drive_name_id, get_full_path_to_file(service, drive_name_id,file), e])
    return size



In [9]:
drive_name_id = ls_drives[1]

In [24]:
def get_size_of_drive(service, drive_name_id):
    ls_files = get_list_of_files_in_drive(service, drive_name_id)
    size = 0
    for file in ls_files[1]:
        try:
            size += int(file)
        except Exception as e:
            pass
    size = size/1024/1024/1024
    return size

In [25]:
def get_size_of_drives(service):
    ls_drives = get_list_of_drives(service)
    ls_size = []
    for drive in ls_drives:
        ls_size.append([drive[0], get_size_of_drive(service, drive)])
    return ls_size

In [26]:
ls_size_drives = get_size_of_drives(service)

In [46]:
drivers = [['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
 ['НКВЗ ремонт АвтоТехники', '0ADgaxv8NdpTkUk9PVA'],
 ['Объекты в работе', '0AOxaPQRooOk2Uk9PVA'],
 ['Остров мечты ППР', '0AImPD4jQdMXJUk9PVA'],
 ['Пекарня', '0AIWIhafOWgmHUk9PVA'],
 ['ПОДРЯДЧИКИ', '0ALnZgMLQFfrtUk9PVA'],
 ['СТО ГБО', '0AF2NtBbx_XvYUk9PVA'],
 ['ТЕХВАГОНМАШ', '0AAAyvFQMbstZUk9PVA'],
 ['Транспроект', '0ADKHIbPu575TUk9PVA'],
 ['ФосАгро', '0AD-4c3pLaBV5Uk9PVA'],
 ['ФРАКДЖЕТ-ТУЛЗ', '0ACjAWTbTqvESUk9PVA'],
 ['Холодный склад', '0ALT2Y1wkG7vCUk9PVA'],
 ['Чайка Лаб', '0APbGeMNQtN3MUk9PVA'],
 ['Читинские ключи', '0AA9gFHffXEitUk9PVA'],
 ['Юрист UA', '0AGg7JoJwa_8nUk9PVA']]

In [47]:
drivers = [['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
]

In [54]:
ls_drives = get_list_of_drives(service)
ls_drives

[['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Ci', '0AHj-PRcbOxSWUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Managment 2', '0AE3Z9Ox2YaurUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['VG.COM', '0AElRg0lTw8iIUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Архив', '0AOP_oKFy4oGJUk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['Винный парк', '0ANhu4lfOrlBiUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Контент_VG', '0AN6chrg4ApnnUk9PVA'],
 ['Контент', '0ANf3LpamTrsTUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
 ['Лобня', '0AIQWy4IyyeaIUk9PVA'],
 ['НКВЗ ремонт АвтоТехники', '0ADgaxv8NdpTkUk9PVA'],
 ['Объединенн

In [42]:
ls_size_drives

[['Юрист UA', 0.0033225594088435173],
 ['Grand_Engineering', 0.029934815131127834],
 ['ТЕХВАГОНМАШ', 0.0414497209712863],
 ['V-CLOUD24', 0.18407616019248962],
 ['Бухгалтер UA', 0.7236725864931941],
 ['Human Resources', 0.8235372556373477],
 ['Адвокатский кабинет', 0.8968739314004779],
 ['ДОУ 360 Выпуск', 0.9078069105744362],
 ['BackOffice', 1.165541491471231],
 ['ПОДРЯДЧИКИ', 1.380760551430285],
 ['Titov Maksim', 1.4614852871745825],
 ['Холодный склад', 1.6076131770387292],
 ['Объекты в работе', 2.2466243552044034],
 ['Читинские ключи', 2.868926392868161],
 ['ФосАгро', 3.8466863138601184],
 ['Лио-Продукт', 4.215722002089024],
 ['Бухгалтер RUS', 5.302054491825402],
 ['Транспроект', 6.9882554449141026],
 ['ФРАКДЖЕТ-ТУЛЗ', 7.858368532732129],
 ['ГипроМез', 8.09878805745393],
 ['СТО ГБО', 9.274343304336071],
 ['Пекарня', 10.64572879858315],
 ['ВДНХ_Колесо обозрения', 10.665728340856731],
 ['Managment 1', 12.608311027288437],
 ['IT-ОТДЕЛ', 14.269819112494588],
 ['ДОУ 360', 16.4524200335145]

In [28]:
ls_size_drives.sort(key=lambda x: x[1])

In [43]:
ls_size_drives[:-10]

[['Юрист UA', 0.0033225594088435173],
 ['Grand_Engineering', 0.029934815131127834],
 ['ТЕХВАГОНМАШ', 0.0414497209712863],
 ['V-CLOUD24', 0.18407616019248962],
 ['Бухгалтер UA', 0.7236725864931941],
 ['Human Resources', 0.8235372556373477],
 ['Адвокатский кабинет', 0.8968739314004779],
 ['ДОУ 360 Выпуск', 0.9078069105744362],
 ['BackOffice', 1.165541491471231],
 ['ПОДРЯДЧИКИ', 1.380760551430285],
 ['Titov Maksim', 1.4614852871745825],
 ['Холодный склад', 1.6076131770387292],
 ['Объекты в работе', 2.2466243552044034],
 ['Читинские ключи', 2.868926392868161],
 ['ФосАгро', 3.8466863138601184],
 ['Лио-Продукт', 4.215722002089024],
 ['Бухгалтер RUS', 5.302054491825402],
 ['Транспроект', 6.9882554449141026],
 ['ФРАКДЖЕТ-ТУЛЗ', 7.858368532732129],
 ['ГипроМез', 8.09878805745393],
 ['СТО ГБО', 9.274343304336071],
 ['Пекарня', 10.64572879858315],
 ['ВДНХ_Колесо обозрения', 10.665728340856731],
 ['Managment 1', 12.608311027288437],
 ['IT-ОТДЕЛ', 14.269819112494588],
 ['ДОУ 360', 16.4524200335145]

In [44]:
ls_size_drives[-10:]

[['VG.COM', 33.50805039424449],
 ['ОСК Шереметьево', 36.87053349241614],
 ['Контент', 43.216087052598596],
 ['Лобня', 45.44711968488991],
 ['Винный парк', 70.22365504875779],
 ['Объединенная пивоварня', 79.06192125100642],
 ['Контент_VG', 82.2075148075819],
 ['Managment 2', 99.93364017829299],
 ['Ci', 159.5339870704338],
 ['Архив', 162.4900345178321]]

In [45]:
ls_size_drives[:15]

[['Юрист UA', 0.0033225594088435173],
 ['Grand_Engineering', 0.029934815131127834],
 ['ТЕХВАГОНМАШ', 0.0414497209712863],
 ['V-CLOUD24', 0.18407616019248962],
 ['Бухгалтер UA', 0.7236725864931941],
 ['Human Resources', 0.8235372556373477],
 ['Адвокатский кабинет', 0.8968739314004779],
 ['ДОУ 360 Выпуск', 0.9078069105744362],
 ['BackOffice', 1.165541491471231],
 ['ПОДРЯДЧИКИ', 1.380760551430285],
 ['Titov Maksim', 1.4614852871745825],
 ['Холодный склад', 1.6076131770387292],
 ['Объекты в работе', 2.2466243552044034],
 ['Читинские ключи', 2.868926392868161],
 ['ФосАгро', 3.8466863138601184]]

In [ ]:
ls_size_drives[:-10]

In [41]:
l1 = [l[0] for l in ls_size_drives[:-10]]
[l for l in ls_drives if l[0] in l1]

[['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
 ['НКВЗ ремонт АвтоТехники', '0ADgaxv8NdpTkUk9PVA'],
 ['Объекты в работе', '0AOxaPQRooOk2Uk9PVA'],
 ['Остров мечты ППР', '0AImPD4jQdMXJUk9PVA'],
 ['Пекарня', '0AIWIhafOWgmHUk9PVA'],
 ['ПОДРЯДЧИКИ', '0ALnZgMLQFfrtUk9PVA'],
 ['СТО ГБО', '0AF2NtBbx_XvYUk9PVA'],
 ['ТЕХВАГОНМАШ', '0AAAyvFQMbstZUk9PVA'],
 ['Транспроект', '0ADKHIbPu575TUk9PVA'],
 ['ФосАгро', '0AD-4c3p

In [37]:
l1 = [l[0] for l in ls_size_drives[-10:]]

In [40]:
l1

['VG.COM',
 'ОСК Шереметьево',
 'Контент',
 'Лобня',
 'Винный парк',
 'Объединенная пивоварня',
 'Контент_VG',
 'Managment 2',
 'Ci',
 'Архив']

In [39]:
[l for l in ls_drives if l[0] in l1]

[['Ci', '0AHj-PRcbOxSWUk9PVA'],
 ['Managment 2', '0AE3Z9Ox2YaurUk9PVA'],
 ['VG.COM', '0AElRg0lTw8iIUk9PVA'],
 ['Архив', '0AOP_oKFy4oGJUk9PVA'],
 ['Винный парк', '0ANhu4lfOrlBiUk9PVA'],
 ['Контент_VG', '0AN6chrg4ApnnUk9PVA'],
 ['Контент', '0ANf3LpamTrsTUk9PVA'],
 ['Лобня', '0AIQWy4IyyeaIUk9PVA'],
 ['Объединенная пивоварня', '0AELFNnmNadJqUk9PVA'],
 ['ОСК Шереметьево', '0ALRUo2DlF1YmUk9PVA']]

In [31]:
ls_drives

[['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Ci', '0AHj-PRcbOxSWUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Managment 2', '0AE3Z9Ox2YaurUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['VG.COM', '0AElRg0lTw8iIUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Архив', '0AOP_oKFy4oGJUk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['Винный парк', '0ANhu4lfOrlBiUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Контент_VG', '0AN6chrg4ApnnUk9PVA'],
 ['Контент', '0ANf3LpamTrsTUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
 ['Лобня', '0AIQWy4IyyeaIUk9PVA'],
 ['НКВЗ ремонт АвтоТехники', '0ADgaxv8NdpTkUk9PVA'],
 ['Объединенн

In [ ]:
corpora='drive',
            driveId=drive_name_id[1],
            includeItemsFromAllDrives=True,
            supportsAllDrives=True,
            fields='nextPageToken, files(id, name)',

In [8]:
# объем диска

def get_size_of_drive(service, drive_id):
    response = service.drives().get(
        driveId=drive_id,
        fields='id, name, size').execute()
    return response['size']

In [9]:
get_size_of_drive(service, ls_drives[1][1])

TypeError: Got an unexpected keyword argument "corpora"

In [13]:
ls_drives[1][1]


'0AHj-PRcbOxSWUk9PVA'

In [15]:
response = service.drives().get(
        driveId= ls_drives[1][1]).execute()

In [16]:
drive_id = ls_drives[1][1]

In [17]:
response = service.drives().get(
        driveId=drive_id,
        fields='name, themeId, backgroundImageLink, backgroundImageFile, capabilities(canAddChildren, canChangeCopyRequiresWriterPermissionRestriction, canChangeDomainUsersOnlyRestriction, canChangeDriveBackground, canChangeDriveMembersOnlyRestriction, canComment, canCopy, canDeleteChildren, canDeleteDrive, canDownload, canEdit, canListChildren, canManageMembers, canReadRevisions, canRename, canRenameDrive, canShare, canTrashChildren, canTrashDrive, canUntrashChildren), colorRgb, createdTime, hidden, id, kind, restrictions(adminManagedRestrictions, copyRequiresWriterPermission, domainUsersOnly, teamDriveMembersOnly), themeId').execute()

response['capabilities']['canReadRevisions']

HttpError: <HttpError 400 when requesting https://www.googleapis.com/drive/v3/drives/0AHj-PRcbOxSWUk9PVA?fields=name%2C+themeId%2C+backgroundImageLink%2C+backgroundImageFile%2C+capabilities%28canAddChildren%2C+canChangeCopyRequiresWriterPermissionRestriction%2C+canChangeDomainUsersOnlyRestriction%2C+canChangeDriveBackground%2C+canChangeDriveMembersOnlyRestriction%2C+canComment%2C+canCopy%2C+canDeleteChildren%2C+canDeleteDrive%2C+canDownload%2C+canEdit%2C+canListChildren%2C+canManageMembers%2C+canReadRevisions%2C+canRename%2C+canRenameDrive%2C+canShare%2C+canTrashChildren%2C+canTrashDrive%2C+canUntrashChildren%29%2C+colorRgb%2C+createdTime%2C+hidden%2C+id%2C+kind%2C+restrictions%28adminManagedRestrictions%2C+copyRequiresWriterPermission%2C+domainUsersOnly%2C+teamDriveMembersOnly%29%2C+themeId&alt=json returned "Invalid field selection canTrashDrive". Details: "Invalid field selection canTrashDrive">

In [11]:
response = service.files().list(
        q="mimeType!='application/vnd.google-apps.folder'",
        corpora='drive',
        driveId=drive_name_id[1],
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,

        fields='nextPageToken, files(id, name, size)').execute()

In [7]:
def f2(name):
    x = bucket.list_blobs(prefix=f'GoogleDriveTransfer/{name}')
    sz = 0
    ls_dt = []
    for blob in x:
        sz += blob.size
        ls_dt.append(blob.updated)
    sz = sz/1024/1024/1024
    df = pd.DataFrame(ls_dt)
    count = len(df)
    return [name, sz, count, df.min(), df.max()]

In [8]:
ls1 = []
for drive in ls_drives:
    ls1.append(f2(drive[0]))

df1 = pd.DataFrame(ls1, columns=['name', 'size', 'count', 'min', 'max'])

In [20]:
response = service.drives().list(pageSize=100).execute()
ids = []
for drive in response['drives']:
    ids.append([drive['name'], drive['id']])

In [6]:
service.close()

In [18]:
ls = get_list_of_drives(service)

In [6]:
[i[0] for i in ls]

['Ci',
 'Адвокатский кабинет',
 'Архив',
 'ВДНХ_Колесо обозрения',
 'Винный парк',
 'ГипроМез',
 'ДОУ 360 Выпуск',
 'ДОУ 360']

In [12]:
credentials_drive = service_account.Credentials.from_service_account_file(
        filename='savvy-concord-374520-bbbb0e3c73c3.json',
        scopes=['https://www.googleapis.com/auth/drive'],
    )
session = requests.AuthorizedSession(credentials_drive)
service = discovery.build('drive', 'v3', credentials=credentials_drive)

In [14]:
page_token = None
response = service.drives().list(
            pageSize=100,
            ).execute()

In [37]:
thr1 = threading.Thread(target=main)
thr1.start()

In [38]:
thr1.is_alive()

False

In [26]:
file_id = ls_files[100]
file_path = get_full_path_to_file(service, drive_ids[0], file_id)
file_path

'GoogleDriveTransfer/Ci/data_db_csv/2022_December/55_2022_12.csv'

In [30]:
blobs = bucket.list_blobs(prefix=file_path)
folders = [blob.name for blob in blobs]

In [32]:
len(folders)

1

In [29]:
folders = [blob.name for blob in blobs]

ValueError: ('Iterator has already started', <google.api_core.page_iterator.HTTPIterator object at 0x000002C3ACD17490>)

In [9]:
ls_files = get_list_of_files_in_drive(service, drive_ids[0])

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    filename='savvy-concord-374520-bbbb0e3c73c3.json',
    scopes=['https://www.googleapis.com/auth/drive'],
)
session = requests.AuthorizedSession(credentials)
service = discovery.build('drive', 'v3', credentials=credentials)

In [ ]:
session.close()



In [10]:
ls_files[100]

'1mYf4uSNIkQnA1aQ_hmwBtYzDVxNs-5Rq'

In [16]:
get_full_path_to_file(service, drive_ids[0], ls_files[100])

'GoogleDriveTransfer/Ci/data_db_csv/2022_December/55_2022_12.csv'

In [ ]:







credentials1 = service_account.Credentials.from_service_account_file(
    filename='credentials.json',
    scopes=['https://www.googleapis.com/auth/devstorage.full_control'],
)
client = storage.Client(credentials=credentials1)
buckets = list(client.list_buckets())
print(buckets)
bucket = client.get_bucket('cabinet_bucket')
blobs = bucket.list_blobs()
#создание папки
blob = bucket.blob('GoogleDriveTransfer/InsertInputCross_1.sql')
blob.upload_from_file(fh)
blob.make_public()
blobs.pages
folders = [blob.name for blob in blobs]

bucket = client.get_bucket('test-bucket-123456')
stream = io.BytesIO(fh.getbuffer())
blob.upload_from_file(stream)

In [5]:
path = 'GoogleDriveTransfer/Ci/1.csv'
blob = bucket.blob(path)

In [6]:
blobs = bucket.list_blobs()

In [8]:
folders = [blob.name for blob in bucket.list_blobs()]

In [9]:
df = pd.DataFrame(folders)
df

,0
0,1/tmp.txt
1,1/trt1.txt
2,10/275/03_Чертеж_планировки_территории_М_1_500...
3,10/275/276/Загрузка_тележки_в_нагревательную_к...
4,10/275/276/Нагревательный_блок_с_возможностью_...
...,...
184502,test5/WebGL/Game1/Game_new.wasm
184503,test5/WebGL/Game2/Running in zavod.data
184504,test5/WebGL/Game2/Running in zavod.framework.js
184505,test5/WebGL/Game2/Running in zavod.loader.js


In [13]:
df1 = df[df[0].str.startswith('GoogleDriveTransfer/')]

In [14]:
df1

,0
3570,GoogleDriveTransfer/BackOffice/Регламенты/desk...
3571,GoogleDriveTransfer/BackOffice/Регламенты/Веде...
3572,GoogleDriveTransfer/BackOffice/Регламенты/Новы...
3573,GoogleDriveTransfer/BackOffice/Регламенты/Прав...
3574,GoogleDriveTransfer/BackOffice/Регламенты/СТП/...
...,...
182533,GoogleDriveTransfer/Юрист UA/Укр Суд/Опекунски...
182534,GoogleDriveTransfer/Юрист UA/Укр Суд/Ответ в о...
182535,GoogleDriveTransfer/Юрист UA/Укр Суд/Прокурату...
182536,GoogleDriveTransfer/Юрист UA/договір коворкінг...


In [15]:
df.to_csv('files_bucket_all.csv')

In [16]:
df1.to_csv('files_bucket_GoogleDriveTransfer.csv')

In [20]:
file_id = ls_files[101]
file_path = get_full_path_to_file(service, drive_ids[0], file_id)

request = service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

Download 100%.


In [21]:
blob = bucket.blob(file_path)


In [22]:
blob.upload_from_string(fh.getvalue())

In [37]:
s_main_folder = 'GoogleDriveTransfer/'
ls3 = get_full_path_to_file(drive_ids[0][1], ls_files[0][1], [])
ls3.reverse()
ls3[0] = drive_ids[0][0]
s_path = s_main_folder + '/'.join(ls3)

In [38]:
s_path

'GoogleDriveTransfer/Ci/data_db_csv/2021_January/71_2021_01.csv'

['Drive', 'data_db_csv', '2021_January', '71_2021_01.csv']

In [138]:
# Список файлов в диске
def get_list_of_files_in_drive2(drive_id):
    response = service.files().list(
        corpora='drive',
        driveId=drive_id,
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        fields='nextPageToken, files(id, name)').execute()
    files1 = response.get('files', [])
    # for file in files:
    # print(file['name'], file['id'], file['mimeType'], file['parents'], file['size'], file['createdTime'], file['modifiedTime'])
    return files1

In [145]:
 response = service.files().list(
    corpora='drive',
    driveId=drive_ids[0],
    includeItemsFromAllDrives=True,
    supportsAllDrives=True,
    fields='nextPageToken, files(id, name)').execute()

In [146]:
files = response.get('files', [])

In [147]:
response = service.files().list(
    corpora='drive',
    driveId=drive_ids[0],
    includeItemsFromAllDrives=True,
    supportsAllDrives=True,
    pageToken=response['nextPageToken'],
    fields='nextPageToken, files(id, name)').execute()

In [148]:
files += response.get('files', [])

In [139]:
fls = get_list_of_files_in_drive(drive_ids[0])

TypeError: Got an unexpected keyword argument "incompleteSearch"

In [79]:
# Список папок в диске
def get_list_of_folders_in_drive(drive_id):
    folders = []
    response = service.files().list(
        corpora='drive',
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        driveId=drive_id,
        q="mimeType='application/vnd.google-apps.folder'",
        fields='nextPageToken, files(id, name)').execute()
    files = response.get('files', [])
    for file in files:
        print(file['name'], file['id'])
        folders.append(file['name'])
    return folders

In [80]:
fds = get_list_of_folders_in_drive(drive_ids[0])

2022_December 1dDXFDiJDxZe1sITq7Qh2STHo6yH3xBUH
2022_November 1dFNzmM2qk9BIJsuMcFFXh5foCji4J-e4
2022_October 1YiojmLMfK4YIHSRWbsQpWKg1nQHLDJfH
2022_September 1TQ1aNTFZtLAfBcpoY83_1N4TG5Sg5e1x
2022_August 1MMHayPtR3lpLpbVKqSd2C45XMaOERUpE
2022_July 1Jgz8BLvjes4QIAqmRZIzUCZIm0k8gCE1
2022_June 1DfzdLYi6haE9fovoeXppYsWVvyBqF0-V
2022_May 189cj8y-QJ1lY1oE5mI3OoNBQgmK7Tt0l
2022_April 134n0mG4LMdF9k3pjF4V9P24rbQr0quwd
2022_March 1yixtMsNRecY4CMqWfmnoelS4p_mFGgQ1
2022_February 1sQMqeXxYTjyj9168VFzZgJ65Z_AHEBK8
2022_January 1ozjOf9KRLgCKn9WubiEvOH9hhIyC2eKG
2021_December 1jXnjjjqErUhr5W3cPK-BzVavpsFfv-qB
2021_November 1e9XKd2_WVYGo8jYTiud6S_ZRlKXPFz6P
2021_October 1a6ivUkBpiCLEO43lNBX6c_PqDaaX7gHb
2021_September 1XjQSITcjuqhXEmwXw9VPyhq0xoTRay2y
2021_August 1Tl_sRACI8vmeNr6P-LBJdeNwZbxuXglk
2021_July 1OJSiA3Bo5zqMJe73QSbu_oQPdZrQnF9E
2021_June 1KvNWRiwr3GG837pAtmzVgq47vIWuCCUB
2021_May 1KYTE3D5ijNkKSY2DdrOW8JtzlJvFhNbt
2021_April 1KVu8-LrtZym7OJlXZcA5h1Np7Y5fZOxb
2021_Mach 1KL3oHd6ShaysScLrfnsEI

In [68]:
'Data Science (Coursera)' in fds

False

In [81]:
# Список папок в папке
def get_list_of_folders_in_folder(drive_id, folder_id):
    response = service.files().list(
        corpora='drive',
        driveId=drive_id,
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        q="mimeType='application/vnd.google-apps.folder' and '{}' in parents".format(folder_id),
        fields='nextPageToken, files(id, name)').execute()
    files = response.get('files', [])
    for file in files:
        print(file['name'], file['id'])

In [82]:
get_list_of_folders_in_folder(drive_ids[0], drive_ids[0])

data_db_csv 1hWb4SgtNk95SmqYW23uh7yr3y-cGI9JD
007 1I2ym5GReGD8cbZTJTApUjZ5ClyUQa73b
Chek 1XCaAYpwepHCURDWZIYIRPCtcHjabhBpA
GoLand 1buUS78znKHryrmHHB28SlqHFLrQo0wlf
Data Science (Coursera) 1G8cpRJIhw-Fsa7c8JptrktScsIPor89G
Опытное тестирование Demo 1KWKAyuskg_w_uuHjAHFLnZBhI-d64JqC
Выпуск 143u95YaWoG4tCkchzg9PgOywZZWEVr4H
MT5 19PktLd9uWtnkFQpa9GDEDIKi9rn18au8
mt45mssql.src 1qZcM4uaYs-XWHPLLgE5hFZ-UJWC7dteo
!MSSQL_Backup 1Fu5ZKM1hoqZVmC4cg0N7sKwF7avOLcvU
Ci 1w1dp2v3-UnBeqR7ChDX6hvR_VTSqMW7B


In [84]:
get_list_of_folders_in_folder(drive_ids[0], '1w1dp2v3-UnBeqR7ChDX6hvR_VTSqMW7B')

MT4_DLL 1lZBx5i9v_6a_d8197tWZPKWvQn1ZkqDp
Experts 1PBBh9FPRnp6DnoWCu6M6zoijtzoXvcD3
Project_MSSQL_DEMO 1LRgHBBcQoypcMthZOGVaa9dpfJ2Fn60D


In [ ]:
from googleapiclient.http import MediaIoBaseDownload
import io


# Скачать файл
def download_file(drive_id, file_id, file_name):
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))
    with open(file_name, 'wb') as f:
        f.write(fh.getbuffer())

In [89]:
from googleapiclient.http import MediaIoBaseDownload
import io

In [110]:
request = service.files().get_media(fileId='10mI1BsOnAY5IyBU3hYr05Jz2cPrBz6_f')
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

Download 100%.


In [92]:
with open('InsertInputCross_1.sql', 'wb') as f:
    f.write(fh.getbuffer())

In [ ]:
# Список файлов в диске
def get_list_of_files_in_drive1(drive_id):
    response = service.files().list(
        corpora='drive',
        driveId=drive_id,
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        fields='nextPageToken, files(id, name, mimeType, parents, size, createdTime, modifiedTime, modifiedByMeTime, owners, shared, starred, trashed)').execute()
    files = response.get('files', [])
    for file in files:
        print(file['name'], file['id'], file['mimeType'], file['parents'], file['size'], file['createdTime'],
              file['modifiedTime'], file['modifiedByMeTime'], file['owners'], file['shared'], file['starred'],
              file['trashed'])

In [87]:
# Список файлов в папке
def get_list_of_files_in_folder(drive_id, folder_id):
    files = []
    response = service.files().list(
        corpora='drive',
        driveId=drive_id,
        includeItemsFromAllDrives=True,
        supportsAllDrives=True,
        q="'" + folder_id + "' in parents",
        fields='nextPageToken, files(id, name)').execute()
    files = response.get('files', [])
    for file in files:
        print(file['name'], file['id'])
        files.append((file['name'], file['id']))
    return files

In [88]:
fls = get_list_of_files_in_folder(drive_ids[0], '1w1dp2v3-UnBeqR7ChDX6hvR_VTSqMW7B')

desktop.ini 1O3yMPs-g6IDxZHOUPTf7rbhFDZOhEbxt
SymbolInfo.xlsx 1pMqFnTIdsSNRl4He9ER7cl10GbW1w5iQ
ТЗ на НУ_ОТВЕТ.docx 11UBbbU4k816iy_XA9Nxhv-yDAWHVtne5
ТЗ на НУ.docx 1qAn9hFqguhMK-V2nggvDuLZIw8PHZ72n
MT5Dll.zip 1nE_S7oMEk9vKT_xPQNyma3i4m9H8E-4n
Drawing1.vsd 1FpI2l9tT1CPrlmLGjn7hiSyDo6fkVret
InsertInputCross_1.sql 10mI1BsOnAY5IyBU3hYr05Jz2cPrBz6_f
InsertInputCross.sql 1O403xSWkNLTeZc7KBQu0Zw9gjxKyLbXz
Project_MSSQL_SOURCE.zip 1VG_db2j1npX54n5837FFTkK3Al1fpdkN
MT4_DLL 1lZBx5i9v_6a_d8197tWZPKWvQn1ZkqDp
Experts 1PBBh9FPRnp6DnoWCu6M6zoijtzoXvcD3
Project_MSSQL_DEMO 1LRgHBBcQoypcMthZOGVaa9dpfJ2Fn60D


TypeError: tuple indices must be integers or slices, not str

In [133]:
# Полный путь к файлу
def get_full_path_to_file(drive_id, file_id):
    response = service.files().get(
        fileId=file_id,
        fields='id, name, parents, mimeType',
        supportsAllDrives=True).execute()
    file = response
    # print(file['name'], file['id'], file['mimeType'], file['parents'])
    print(file['name'])
    if file.get('parents') is not None:
        for parent in file['parents']:
            get_full_path_to_file(drive_id, parent)

In [135]:
get_full_path_to_file(drive_ids[0], '1surnjzxcGhE_azRpjmVY2_GsiBWeJdPx')

71_2021_01.csv
2021_January
data_db_csv
Drive


In [ ]:
# Создание папки
def create_folder(folder_name, parent_id):
    file_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_id]
    }
    file = service.files().create(body=file_metadata,
                                  fields='id').execute()
    print('Folder ID: %s' % file.get('id'))

In [35]:
results = service.files().list(
    fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

In [36]:
if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))

Files:
Бренд бук (1lA813gD-q-Cp8B19KVWke_1LQ1SwCDMr)
Шаблон 1 - светлый фон.docx (13o3cBN4jf8DSE3aH-dgUfEGjjI6yGFda)
Шаблон 1 - светлый фон.pdf (1Q_6uZtHPhQFoi5xp_NNJPrRtjI9sAuTb)
Шаблон 2 - светлый фон.pdf (1SGlTDRVaG4sT9iL3GqYVJiENcH6H7j3n)
Шаблон 2 - светлый фон.docx (1vmCqosN50VTzJRzX_g2UpkutDZ2BXiey)
Шаблон - темный фон.pdf (1T6cRNPcWR1l52QMswB7uc9Bl0bWLtzzL)
Шаблон- темный фон.docx (1dE2rBfUUpdnpCbfrJMypLThcvHwriwZw)
Визитка (1MQNg0lrgugPSjFWOXxKQ09lST090Rrhz)
Реклама для раздачи (1Cd5khZUrjiPXdwIF3xK4w_DQEStdBPQH)
Бланк КП (12ExvKVLD8Ng-WdGhxf8M3-q3iOYwMsE5)
Шаблоны шапки для коммерческого (18eJlLBs5Z_nzywHm_D9rcxyVMWoIhBKR)
GRAND ENGINEERING (BIM).pdf (1tkCvqjq3t2vtvKvY_iD4WYYHRLGNIRrX)
V-GRAND_reference list_03.2022.docx (1GT9Bc9qLRskzqvJ6W-BjeFtGTK_XyJyS)
подпись в почте.docx (1PUoVhlbcyasJrBcanFCI1XmeCmFuL55a)
Этапы (1f8A4t885GE9LkGrETUvEOWSOQhoZMSWugp9cDlZaoPs)
Презентация (1xLC0b5HiZFVKxD_bU9q5Rz4CLbWDC17D)
Перечень выполненных объектов GRAND ENGINEERING.docx (1yTcSplXli0F

In [93]:
from google.cloud import storage

In [117]:
credentials1 = service_account.Credentials.from_service_account_file(
    filename='credentials.json',
    scopes=['https://www.googleapis.com/auth/devstorage.full_control'],
)

In [118]:
client = storage.Client(credentials=credentials1)

In [119]:
buckets = list(client.list_buckets())

In [120]:
print(buckets)

[<Bucket: artifacts.grand-358005.appspot.com>, <Bucket: cabinet_bucket>]


In [121]:
bucket = client.get_bucket('cabinet_bucket')

In [122]:
blobs = bucket.list_blobs()

In [123]:
#создание папки
blob = bucket.blob('GoogleDriveTransfer/InsertInputCross_1.sql')

In [124]:
blob.upload_from_file(fh)

ValueError: Stream must be at beginning.

In [114]:
blob.make_public()

BadRequest: 400 GET https://storage.googleapis.com/storage/v1/b/cabinet_bucket/o/GoogleDriveTransfer%2F/acl?prettyPrint=false: Cannot get legacy ACL for an object when uniform bucket-level access is enabled. Read more at https://cloud.google.com/storage/docs/uniform-bucket-level-access

In [104]:
blobs.pages

ValueError: ('Iterator has already started', <google.api_core.page_iterator.HTTPIterator object at 0x00000141DA9D7FA0>)

In [107]:
folders = [blob.name for blob in blobs]

In [ ]:
bucket = client.get_bucket('test-bucket-123456')

In [127]:
stream = io.BytesIO(fh.getbuffer())

In [128]:
blob.upload_from_file(stream)

In [ ]:
import os

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credentials.json'

In [ ]:
from google.cloud import storage

In [ ]:
storage_client = storage.Client()

In [ ]:
buckets = list(storage_client.list_buckets())

In [ ]:
print(buckets)

In [ ]:
bucket = storage_client.get_bucket('my-bucket')

In [ ]:
blob = bucket.blob('my-blob')

In [ ]:
blob.upload_from_string('Hello, World!')

In [ ]:
blob.download_as_string()

In [ ]:
blob.make_public()

In [ ]:
blob.public_url

In [ ]:
blob.delete()


In [ ]:
from google.cloud import storage

In [ ]:
storage_client = storage.Client()

In [ ]:
bucket = storage_client.get_bucket('my-bucket')

In [ ]:
blob = bucket.blob('my-blob')

In [ ]:
blob.upload_from_filename(filename='local/path/to/file.txt')

In [ ]:
blob.download_to_filename(filename='local/path/to/file.txt')

In [ ]:
blob.delete()


In [ ]:
from google.cloud import storage

In [ ]:
storage_client = storage.Client()

In [ ]:
bucket = storage_client.get_bucket('my-bucket')

In [ ]:
blob = bucket.blob('my-blob')

In [ ]:
blob.upload_from_file(file_obj=my_file)

In [ ]:
blob.download_to_file(file_obj=my_file)

In [ ]:
blob.delete()


In [ ]:
from google.cloud import storage

In [ ]:
storage_client = storage.Client()

In [ ]:
bucket = storage_client.get_bucket('my-bucket')

In [ ]:
blob = bucket.blob('my-blob')

In [ ]:
blob.upload_from_filename(filename='local/path/to/file.txt')

In [ ]:
blob.download_to_filename(filename='local/path/to/file.txt')

In [ ]:
blob.delete()


In [ ]:
from google.cloud import storage

In [ ]:
storage_client = storage.Client()

In [ ]:
bucket = storage_client.get_bucket('my-bucket')

In [ ]:
blob = bucket.blob('my-blob')

In [ ]:
blob.upload_from_file(file_obj=my_file)

In [ ]:
blob.download_to_file(file_obj=my_file)

In [ ]:
blob.delete()


In [ ]:
from google.cloud import storage

In [ ]:
storage_client = storage.Client()

In [ ]:
bucket = storage_client.get_bucket('my-bucket')

In [ ]:
blob = bucket.blob
pageSize = 10,
fields = "nextPageToken, files(id, name, mimeType, parents, createdTime)",
q = filter).execute()